# Start systems

For a given polynomial system there are *infinitely many* possible start systems and homotopies.
We say that a homotopy is *optimal* if $H(x,1)$ has the same number of solutions as $F(x)=H(x,0)$. Optimal homotopies only need to track the minimal number of paths to  get *all* solutions of $F(x)$. But constructing optimal homotopies is in general *not* possible. We don't even know an efficient method to compute the number of solutions of $F(x)$. 

Instead of aiming at optimal homotopies, we construct homotopies $H(x,t)$ such that the number of solutions of the start system $H(x,1)$ is greater or equal than the number of solutions of the target $H(x,0)$.


## Totaldegree homotopy


Let $F=(f_1,\ldots,f_n)$ be a system of $n$ polynomials in $n$ variables $x=(x_1,\ldots,x_n)$. By default `HomotopyContinuation.jl` uses the so called totaldegree homotopy for solving $F(x)=0$. I.e.

```julia
solve(F)
```

performs a totaldegree homotopy. This works as follows. Suppose that the $i$-th polynomial has degree $d_i$. Then the totaldegree homotopy for $F$ is

$$H(x,t) = tG(x) + (1-t)F(x), \text{ where } G(x) = \gamma\,\begin{bmatrix}x_1^{d_1}-1\\\ \vdots\\\ x_n^{d_n}-1\end{bmatrix}$$

and where $\gamma$ is a random complex number. The zeros of $G(x)$ are given by combinations of roots of unity. There are $d_1\cdots d_n$ many of them. By Bezout's theorem $F(x)=0$ has at most  $d_1\cdots d_n$ solutions. In particular, the totaldegree homotopy is optimal for a generic system $F(x)$.

For instance, let $F=(f_1,f_2)$ with

$$    f_1(x,y) = (x^4 + y^4 - 1)(x^2 + y^2 - 2) + x^5y  \quad \text{ and } \quad f_2(x,y) =  x^2 + 2xy^2 - 2y^2 - \frac{1}{2} \,.$$

The polynomial $f_1$ has degree $6$ and the polynomial $f_2$ has degree 3. Now [Bezout's theorem](https://en.wikipedia.org/wiki/Bézout%27s_theorem) tells us that such a polynomial system has at most $6 \cdot 3=18$ isolated solutions. We then can construct the polynomial system

$$G(x,y) = \begin{bmatrix} x^2 - 1 \\ y^2 - 1\end{bmatrix}$$

which has the $18$ solutions
$(\exp(i 2\pi\frac{k_1}{6}), \;\exp(i 2\pi\frac{k_2}{3})$
where $k_1 \times k_2 \in \{0,\ldots,5\} \times \{0,1,2\}$.

Let us implement this in Julia.

In [ ]:
using HomotopyContinuation
@polyvar x y
f1 = (x^4 + y^4 - 1)*(x^2 + y^2 - 2) + x^5*y
f2 = x^2 + 2x*y^2 - 2y^2 - 1/2

F = [f1; f2]
G = [x^6 - 1; y^3 - 1]

# We use map the cartesian product of 0:5 and 0:2 to the solutions of G
sols_G = map(k -> [exp(im * 2 * pi * k[1] / 6); exp(im * 2 * pi * k[2] / 3)], Iterators.product(0:5,0:2))

Now, we can track `sols_G` from `G` to `F` as follows.

In [ ]:
solve(G, F, sols_G)

The last command is equivalent to the following two

In [ ]:
solve(F, start_system = :total_degree)

or

In [ ]:
solve(F)

Before doing a totaldegree homotopy it is often insightful to check the number of paths that need to be tracked in total:

In [ ]:
bezout_number(F)

## Polyhedral homotopy

A bit more involved is the polyhedral homotopy. This homotopy uses the Newton polytopes of the $f_i$ to generate a start system. The polyhedral homotopy is optimal for a generic system with the same sparsity structure as $F$. Here is how it works.

In [ ]:
solve(F, start_system = :polyhedral) 

As before, we can consider the number of paths that need to be tracked by the polyhedral homotopy beforehand.

In [ ]:
mixed_volume(F)

## Multiprojective totaldegree

Sometimes, the variables can be grouped to obtain a better start system.

A generalization of Bezout's theorem by [Shafarevich](https://en.wikipedia.org/wiki/Multi-homogeneous_Bézout_theorem) works over products of projective spaces. We can exploit this theorem to generate start systems.

In [ ]:
solve(F, variable_groups = [[x], [y]])

(In this example it is actually worse to use the grouping $\{x\}, \{y\}$. In the next notebook we will see an example, where it is better.)

## Special homotopies

In particular situations it usually most efficient to compute a start system, which is adapted to the problem. We will see this in action when we deal with parameter homotopies.